This script fetches live ADS-B data from FlightRadar24.com. Getting historical data is harder than getting live data. But getting live data is slow.

In [1]:
import subprocess
import time
import json
import pandas as pd
import csv

In [ ]:
start_time = time.time()
out_file = open('../data/ads-b_extra.csv', 'a')
writer = csv.writer(out_file)
command = ['curl', 'https://data-live.flightradar24.com/zones/fcgi/feed.js?bounds=45,35,-78,-68&faa=1&mlat=1&flarm=1&adsb=1&gnd=1&air=1&vehicles=1&estimated=1&maxage=7200&gliders=1&stats=1']
curr_time = time.time()
total_time = curr_time - start_time

run_time_hours = 336 # 2 weeks * 7 days per week * 24 hours per day
query_wait_time = 8
ignore_keys = {"full_count", "version", "stats"}

while total_time < (run_time_hours * 60 * 60):
    try:
        out = subprocess.run(command, capture_output = True)
        out = json.loads(out.stdout)
        out = {key: out[key] for key in out.keys() if key not in ignore_keys}
        df = pd.DataFrame(out).T.reset_index(drop = False)
        
        for i in range(df.shape[0]):
            row = df.iloc[i]
            writer.writerow([curr_time] + row.tolist())
        
        curr_time = time.time()
        total_time = curr_time - start_time
        time.sleep(query_wait_time)
    except:
        continue

In [ ]:
out_file.close()